# Introduction and Schema Diagram¶

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Overview of the Data¶

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [22]:
%%sql
WITH c_info AS ( 
    SELECT
    customer_id,
    SUM(total) total
FROM invoice
GROUP BY customer_id
ORDER BY customer_id
), max_buyer AS (
    SELECT 
    customer_id,
    MAX(total) total
FROM c_info
)

SELECT 
    *
FROM customer
INNER JOIN max_buyer ON customer.customer_id = max_buyer.customer_id;



Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id,customer_id_1,total
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4,5,144.54000000000002


# Selecting New Albums to Purchase¶

In [60]:
%%sql
WITH cus_usa AS 
    (
    SELECT 
        c.customer_id customer_id,
        c.country country,
        i.invoice_id invoice_id,
        il.track_id track_id,
        g.name name
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    INNER JOIN track t ON il.track_id = t.track_id
    INNER JOIN genre g ON t.genre_id = g.genre_id
    WHERE c.country = "USA"
    )

SELECT 
    name,
    COUNT(*) track_sold,
    ROUND(CAST(COUNT(*) AS FLOAT) / (
                SELECT 
                COUNT(*)
                FROM cus_usa
    ), 3) percentage_sold
FROM cus_usa
GROUP BY name
ORDER BY 2 DESC;


Done.


name,track_sold,percentage_sold
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Latin,22,0.021
Pop,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:
- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

It's good to note that combined, these three genres only make up only 17% of total sales, so we should pay attention to artists and albums from the 'rock' genre, which accounts for 53% of sales.

# Analyzing Employee Sales Performance

In [67]:
%%sql
WITH sales_per_customer AS (
    SELECT 
            c.*,
            i.total total
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
)

SELECT 
        e.first_name || " " || e.last_name name,
        e.hire_date hire_date,
        SUM(spc.total) total
FROM employee e
INNER JOIN sales_per_customer spc ON spc.support_rep_id = e.employee_id
GROUP BY 1
ORDER BY total DESC;

Done.


name,hire_date,total
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


From the sales support agent data, we can notice that the earlier employee was hired the more sales he/she made.

# Analyzing Sales by Country¶

In [17]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )
    

SELECT 
        country,
        customers,
        total_sales,
        average_order,
        customer_lifetime_value
FROM (
    SELECT
        country,
        COUNT(DISTINCT customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / COUNT(DISTINCT customer_id) customer_lifetime_value,
        SUM(unit_price) / COUNT(DISTINCT invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort, average_order DESC
     );


Done.


country,customers,total_sales,average_order,customer_lifetime_value
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


Based on the data, we see that following countries have potential to grow:
    - Czech Republic;
    - United Kingdom;
    - India;
But we need to be cautious before running any major campaign since sample size is too small.

# Albums vs Individual Tracks¶

In [59]:
%%sql
WITH 
    invoice_first_track AS 
    (
    SELECT 
            invoice_id,
            MIN(track_id) track_id
    FROM invoice_line
    GROUP BY 1
    )
SELECT 
        album_purchase,
        COUNT(album_purchase) number,
        CAST(COUNT(album_purchase) AS FLOAT) / (SELECT COUNT(*)
                                             FROM invoice) percentage 
FROM (
        SELECT 
            CASE
                WHEN (
                    SELECT track_id FROM track
                    WHERE album_id = (
                                        SELECT album_id FROM track
                                        WHERE track_id = ifs.track_id
                                    )
                    EXCEPT
                    SELECT track_id FROM invoice_line
                    WHERE invoice_id = ifs.invoice_id
                    ) IS NULL
                    AND
                    (
                    SELECT track_id FROM invoice_line
                    WHERE invoice_id = ifs.invoice_id
                    EXCEPT
                    SELECT track_id FROM track
                    WHERE album_id = (
                                        SELECT album_id FROM track
                                        WHERE track_id = ifs.track_id
                                    )
                    ) IS NULL
            THEN "yes"
            ELSE "no"
        END AS album_purchase
        FROM invoice_first_track ifs
     )
GROUP BY album_purchase;



Done.


album_purchase,number,percentage
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchase is only 18.6 percent. The Chinook should keep buying selected individual songs from albums rather than whole albums. 

# Popular artists in playlists

In [89]:
%%sql
WITH 
    play_info AS 
    (
    SELECT
            ar.name artist,
            COUNT(DISTINCT p.name) number
            
    FROM playlist p
    INNER JOIN playlist_track pl ON pl.playlist_id = p.playlist_id
    INNER JOIN track t ON t.track_id = pl.track_id
    INNER JOIN album al ON al.album_id = t.album_id
    INNER JOIN artist ar ON al.artist_id = ar.artist_id
    GROUP BY  artist
    )

SELECT *
FROM play_info
ORDER BY 2 DESC
LIMIT 10;



Done.


artist,number
Eugene Ormandy,6
Academy of St. Martin in the Fields & Sir Neville Marriner,5
Berliner Philharmoniker & Herbert Von Karajan,5
English Concert & Trevor Pinnock,5
The King's Singers,5
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",4
Adrian Leaper & Doreen de Feis,4
Alberto Turco & Nova Schola Gregoriana,4
Antal Doráti & London Symphony Orchestra,4
Barry Wordsworth & BBC Concert Orchestra,4


Eugene Ormandy is the most popular artist in playlists. The different kinds of playlists represent the versatility in the kind of music produced by an artist.

Let's see if the most popular artist in playlists is the best selling artist for Chinook. To do this, we would need invoice_line, track, album, and artist tables (refer schema).

In [96]:
%%sql
SELECT
    ar.name artist_name,
    COUNT(*) no_of_tracks,
    g.name genre
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON al.artist_id = ar.artist_id
INNER JOIN genre g ON t.genre_id = g.genre_id 
GROUP BY artist_name
ORDER BY no_of_tracks DESC
LIMIT 10;

Done.


artist_name,no_of_tracks,genre
Queen,192,Rock
Jimi Hendrix,187,Rock
Nirvana,130,Rock
Red Hot Chili Peppers,130,Rock
Pearl Jam,129,Rock
AC/DC,124,Rock
Guns N' Roses,124,Metal
Foo Fighters,121,Rock
The Rolling Stones,117,Rock
Metallica,106,Metal


The sales are mostly dominated by rock bands, which is not surprising based on query 1. Popularity is a metric, and can be interpreted in different ways based on the information need. As far as sales are concerned, genre seems to affect the sales more than versatility/ playlists.

# Protected vs Non-protected media types¶

In [20]:
%%sql
WITH 
    track_media AS
    (
    SELECT 
        il.track_id,
        mt.name media_type
    FROM invoice_line il
    INNER JOIN track t ON t.track_id = il.track_id
    INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id 
    ),
    
    protected AS 
    (
    SELECT *,
        CASE
            WHEN media_type LIKE "%Protected%" THEN "yes"
            ELSE "no"
        END AS protected
    FROM track_media
    )
    
SELECT 
    protected,
    COUNT(DISTINCT track_id) number,
    ROUND(CAST(COUNT(DISTINCT track_id) AS FLOAT) * 100 / (SELECT COUNT(DISTINCT track_id)
                                                FROM invoice_line), 2) percent
FROM protected
GROUP BY 1

Done.


protected,number,percent
no,1652,91.47
yes,154,8.53


Out of all the unique tracks sold (1806 tracks) by the store, ~91% comprises of unprotected media.